### Setup IQS Connection

In [ ]:
import iqs_jupyter
iqs = iqs_jupyter.connect(
    address='127.0.0.1:47700',
    user='admin',
    password='mypassword'
)

In [ ]:
# one can already invoke operations on the server, e.g.
iqs.repository.update_repository()

### Select and load TWC Revision

In [ ]:
revision_selector = iqs.jupyter_tools.twc_revision_selector_widget()

In [ ]:
iqs.persistent_index.index_revision(revision_selector.value())

In [ ]:
iqs.in_memory_index.load_revision(revision_selector.value())

### Register and Execute Query

In [ ]:
iqs.queries.unregister_all_queries()

In [ ]:
iqs.queries.register_queries_plain_text('''
pattern dependencies(source : NamedElement, target : NamedElement) {
  Dependency.supplier(dependency, target); Dependency.client(dependency, source);
}
''', query_package="iqs4twc.queries.example")

In [ ]:
qResults = iqs.query_execution.execute_query({
  "revisionDescriptor": revision_selector.value(),
  "queryFQN": "iqs4twc.queries.example.dependencies"
})

### Process query results

In [ ]:
qResults

In [ ]:
qResults.to_list_of_matches()

In [ ]:
first_result_element = qResults.to_list_of_matches()[0]['source']
first_result_element # check element hover for element descriptor details

In [ ]:
some_element = revision_selector.value().get_element_descriptor_by_id("c1be5a8c-a89e-40aa-87a8-8f7ef9527292") # can also look up directly
some_element

In [ ]:
from iqs_jupyter import binding

qResults_restricted = iqs.query_execution.execute_query({
  "revisionDescriptor": revision_selector.value(),
  "queryFQN": "iqs4twc.queries.example.dependencies",
  "parameterBinding": binding(target=some_element)
})
qResults_restricted

### Convert query results to Python/Jupyter-friendly Pandas dataframes and process them as such

In [ ]:
qResults_df = qResults.to_data_frame()
qResults_df

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
qResults_df.applymap(lambda element: element.element_id).source.value_counts().iplot(
    kind='bar', filename='cufflinks/categorical-bar-chart/frequent_sources',
    yTitle='Number of Dependencies', title='Dependencies from Source',
)

### Connect to OSMC and browse element details

In [ ]:
osmc_connector = iqs_jupyter.OSMCConnectorWidget()

In [ ]:
osmc = osmc_connector.connect()

In [ ]:
element_info = osmc.show_element_info_widget(first_result_element)
element_info.display()

In [ ]:
element_info.selected_element()

### Sandbox